In [1]:
%cd /home/dongmin/23FW-NCG/jeongganbo-omr
%load_ext autoreload
%autoreload 2

/home/dongmin/userdata/jeongganbo-omr


In [2]:
from time import time
import csv
import re
import glob
import json
from random import randint, choice, uniform, seed
from collections import defaultdict
import pickle
from pathlib import Path

from tqdm import tqdm
import matplotlib.pyplot as plt

from omegaconf import OmegaConf, DictConfig

import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader

from exp_utils import JeongganSynthesizer, get_img_paths
from exp_utils.model_zoo import OMRModel, TransformerOMR
from exp_utils.train_utils import Trainer, Dataset, Tokenizer, pad_collate, LabelStudioDataset, draw_low_confidence_plot

dprint = lambda d: print(json.dumps(d, indent=2))

/home/dongmin/.local/share/virtualenvs/jeongganbo-omr-QQMBMPoZ/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
from exp_utils import SYMBOL_W_DUR_EN_LIST, JeongganSynthesizer

len(SYMBOL_W_DUR_EN_LIST), len(JeongganSynthesizer.get_pitch_range())

p_prob = [1] * 12 + [5, 1] + [0.5] * len(SYMBOL_W_DUR_EN_LIST)
np.asarray(p_prob) / sum(p_prob)

array([0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429,
       0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429,
       0.03571429, 0.03571429, 0.17857143, 0.03571429, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714])

In [43]:
sum([0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429,0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429,0.03571429, 0.03571429]), \
sum([0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714])

(0.42857147999999995, 0.3571427999999999)

## pipline mock-up

### load model and inference

In [3]:
# random seed setting
project_root_dir = Path('.')
output_dir = project_root_dir / 'outputs'

dir_date = '2024-05-04'
dir_time = '20-31-38'

exp_dir = output_dir / dir_date / dir_time

conf = OmegaConf.load(exp_dir / '.hydra' / 'config.yaml')
device = torch.device(conf.general.device)

model_dir = exp_dir / 'model'

print('\nload tokenizer')

tokenizer_vocab_fn = model_dir / f'{conf.general.model_name}_tokenizer.txt'
tokenizer = Tokenizer(vocab_txt_fn=tokenizer_vocab_fn)

print('COMPLETE: load tokenizer')


print('\ndata_set loading...')

test_set = LabelStudioDataset(project_root_dir / conf.data_path.test, project_root_dir / 'jeongganbo-png/splited-pngs', remove_borders=conf.test_setting.remove_borders, is_valid=True)

test_set.tokenizer = tokenizer

test_loader = DataLoader(test_set, batch_size=1000, shuffle=False, collate_fn=pad_collate, num_workers=conf.dataloader.num_workers_load)

print('COMPLETE: data_set loading')


print('\nmodel initializing...')
model = TransformerOMR(conf.model.dim, len(tokenizer.vocab), enc_depth=conf.model.enc_depth, dec_depth=conf.model.dec_depth, num_heads=conf.model.num_heads, dropout=conf.model.dropout)
model.load_state_dict(torch.load(model_dir / f'{conf.general.model_name}_HL_{conf.test_setting.target_metric}_best.pt', map_location='cpu')['model'])

tester = Trainer(model, 
                 None, #optimizer
                 None, #loss_fn
                 None, #train_loader
                 test_loader, 
                 tokenizer,
                 device=device, 
                 scheduler=None,
                 aux_loader=None,
                 aux_freq=None,
                 mix_aux=None,
                 aux_valid_loader=None,
                 wandb=None, 
                 model_name=conf.general.model_name,
                 model_save_path=model_dir,
                 checkpoint_logger=None)

print('COMPLETE: model initializing')


print('\nTesting...')

_, _, pred_tensor_list, confidence_tensor_list = tester.validate(with_confidence=True)

print('COMPELETE: Testing')


load tokenizer
COMPLETE: load tokenizer

data_set loading...
COMPLETE: data_set loading

model initializing...
COMPLETE: model initializing

Testing...


COMPELETE: Testing


### process test result

In [4]:
pred_list = []

for b_idx in range(len(pred_tensor_list)):
  pred_list += pred_tensor_list[b_idx].tolist()

pred_list = list( enumerate(pred_list) )

In [5]:
filter_specials = lambda seq: [ x for x in seq if x not in (0, 1, 2) ]

def calc_acc(gt, prd):
  gt, prd = torch.tensor(gt, dtype=torch.long), torch.tensor(prd, dtype=torch.long)
  
  if prd.shape[0] > gt.shape[0]:
    prd = prd[:gt.shape[0]]
  elif prd.shape[0] < gt.shape[0]:
    prd = torch.cat([prd, torch.zeros(gt.shape[0] - prd.shape[0])], dim=-1)
  
  num_tokens = gt.shape[0]
  num_match_token = prd == gt
  num_match_token = num_match_token[gt != 0].sum().item()
  
  return num_match_token / num_tokens

def make_gt_pred_tuple_list(_tokenizer, ls, with_img=False):
  tup_list = []
  
  for didx, pred in ls:
    img, label = test_set.get_item_by_idx(didx)
    
    if not with_img:
      img = None
    
    label_enc = _tokenizer(label)[1:-1]
    pred = filter_specials(pred)
    
    acc = calc_acc(label_enc, pred)
    
    tup_list.append( (didx, img, label_enc, pred, acc) )
  
  return tup_list

pred_list = make_gt_pred_tuple_list(tokenizer, pred_list)

In [6]:
pred_list_long = list( filter( lambda x: len(x[2]) > 2, pred_list ) )

len(pred_list_long)

1281

In [7]:
pred_list_long = sorted(pred_list_long, key=lambda x: x[4])

### pick test cases

In [8]:
seed(1)

test_ls = []

for i in range(11):
  f = [t for t in pred_list_long if t[4] < (i + 1)*0.1 and t[4] > i*0.1]
  test_ls += [choice(f)] if len(f) > 0 else []

len(test_ls)

10

In [9]:
test_ls

[(402,
  None,
  [119, 34, 12, 3, 4, 14, 3, 89, 16, 3, 121, 18],
  [119, 12, 3, 4, 14, 3, 89, 16, 3, 121, 18],
  0.08333333333333333),
 (1218,
  None,
  [76, 52, 6, 3, 127, 58, 7],
  [76, 6, 3, 127, 58, 7],
  0.14285714285714285),
 (1333,
  None,
  [140, 12, 3, 4, 15, 3, 127, 49, 18],
  [140, 6, 3, 127, 49, 7],
  0.2222222222222222),
 (1356, None, [140, 55, 15], [140, 15], 0.3333333333333333),
 (370,
  None,
  [80, 12, 3, 4, 14, 3, 87, 57, 16, 3, 83, 18],
  [80, 57, 12, 3, 4, 14, 3, 87, 16, 3, 83, 18],
  0.4166666666666667),
 (1494,
  None,
  [140, 12, 3, 4, 57, 15, 3, 106, 18],
  [140, 57, 12, 3, 4, 15, 3, 106, 18],
  0.5555555555555556),
 (951, None, [140, 67, 15], [140, 68, 15], 0.6666666666666666),
 (1247,
  None,
  [127, 12, 3, 4, 15, 3, 4, 17, 3, 141, 47, 19],
  [127, 12, 3, 4, 15, 3, 4, 17, 3, 87, 19],
  0.75),
 (1529,
  None,
  [4, 12, 3, 4, 57, 15, 3, 127, 18],
  [4, 12, 3, 4, 57, 15, 3, 127, 64, 18],
  0.8888888888888888),
 (1347,
  None,
  [105, 12, 3, 4, 14, 3, 127, 16, 3, 

## New metrics
* NER: note error rate
* PER: position error rate
* NPER: note-position pair error rate

In [12]:
def get_notes_and_positions(label):
  pattern = r'([^_\s:]+|_+[^_\s:]+|:\d+|[-])'
  
  def clean_findings(fds):
    int_idx = []
    
    for i, t in enumerate(fds):
      try:
        int(t[1:]) # if str p cannot be cast into integer goto except block
        int_idx.append(i)
      except:
        pass
    
    if len(int_idx) < 1:
      return [fds]
    
    split_fds = []
    start_idx = 0
    
    for i in int_idx:
      end_idx = i + 1
      split_fds.append(fds[start_idx:end_idx])
      start_idx = end_idx
    
    return split_fds

  token_groups = label.split()

  notes = []
  positions = []
  ornaments = []

  for group in token_groups:
    findings = re.findall(pattern, group)
    findings = clean_findings(findings)
    
    for fd in findings:
      note = fd[0]
      position = fd[-1]
      ornament = fd[1:-1]
      
      if note[0] == '_':
        ornament = [note] + ornament
        note = '<pad>'
        
      notes.append(note)
      positions.append(position[1:])
      ornaments.append(ornament)

  return notes, positions, ornaments

for test_case in test_ls:
  label, pred = test_case[2:4]
  
  label_dec = tokenizer.decode( label )
  pred_dec = tokenizer.decode( pred )
  
  label_note, label_pos, label_ornament = get_notes_and_positions(label_dec)
  pred_note, pred_pos, pred_ornament = get_notes_and_positions(pred_dec)
  
  label_note_pos = list( zip(label_note, label_pos) )
  pred_note_pos = list( zip(pred_note, pred_pos) )
  
  map_ornamnet_pos = lambda ol, pl: [(o, p) for i, p in enumerate(pl) for o in ol[i]]
  map_ornament_note_pos = lambda ol, tl: [ (o, *t) for i, t in enumerate(tl) for o in ol[i]]
  
  print(label_dec)
  print(map_ornamnet_pos(label_ornament, label_pos))
  print(map_ornament_note_pos(label_ornament, label_note_pos))
  
  print()

청남_노네:2 -:4 니나*:6 청임:8
[('_노네', '2')]
[('_노네', '청남', '2')]

고_덧길이표:10 태_반길이표:11
[('_덧길이표', '10'), ('_반길이표', '11')]
[('_덧길이표', '고', '10'), ('_반길이표', '태', '11')]

황:2 -:5 태_니레:8
[('_니레', '8')]
[('_니레', '태', '8')]

황_루러표:5
[('_루러표', '5')]
[('_루러표', '황', '5')]

노:2 -:4 니_미는표:6 느나:8
[('_미는표', '6')]
[('_미는표', '니', '6')]

황:2 -_미는표:5 배중:8
[('_미는표', '5')]
[('_미는표', '-', '5')]

황_자출:5
[('_자출', '5')]
[('_자출', '황', '5')]

태:2 -:5 -:7 흘림표_니:9
[('_니', '9')]
[('_니', '흘림표', '9')]

-:2 -_미는표:5 태:8
[('_미는표', '5')]
[('_미는표', '-', '5')]

배임:2 -:4 태:6 배임:7 황_루러표:9
[('_루러표', '9')]
[('_루러표', '황', '9')]



In [13]:
test_lbs = [
  '임_노니로:1 중:3 -:5 무_느니-르_노니로:8_느니-르_노니로:8',
  '황:1 태:3 -:5 -:7 배임_노니로:9_노니로:9',
  '황:1 태:3 -_흘림표:5 -:8 니:9니:9',
  '태:1 황:3 -:4 니:6 배남_흘림표:8_흘림표:8',
]

for lb in test_lbs:
  print(lb)
  print(get_notes_and_positions(lb))

# print(test_lbs[2])
# get_notes_and_positions(test_lbs[2])

임_노니로:1 중:3 -:5 무_느니-르_노니로:8_느니-르_노니로:8
(['임', '중', '-', '무', '<pad>'], ['1', '3', '5', '8', '8'], [['_노니로'], [], [], ['_느니-르', '_노니로'], ['_느니-르', '_노니로']])
황:1 태:3 -:5 -:7 배임_노니로:9_노니로:9
(['황', '태', '-', '-', '배임', '<pad>'], ['1', '3', '5', '7', '9', '9'], [[], [], [], [], ['_노니로'], ['_노니로']])
황:1 태:3 -_흘림표:5 -:8 니:9니:9
(['황', '태', '-', '-', '니', '니'], ['1', '3', '5', '8', '9', '9'], [[], [], ['_흘림표'], [], [], []])
태:1 황:3 -:4 니:6 배남_흘림표:8_흘림표:8
(['태', '황', '-', '니', '배남', '<pad>'], ['1', '3', '4', '6', '8', '8'], [[], [], [], [], ['_흘림표'], ['_흘림표']])


In [14]:
def split_enc_seq(_tokenizer, enc_lb):
  blank_tok_idx = _tokenizer.tok2idx[' ']
  split_idx_list = [ i for i, t in enumerate(enc_lb) if t == blank_tok_idx ]
  
  is_ornament = lambda ti: '_' == _tokenizer.vocab[ti][0]
  is_position = lambda ti: ':' == _tokenizer.vocab[ti][0]
  
  def clean_findings(gl):
    il = [ i for i, t in enumerate(gl) if is_position(t) ]
    
    ls = []
    st = 0
    
    for i in il + [None]:
      ed = i + 1 if i else None
      
      sl = gl[st:ed]
      
      if len(sl) < 1:
        continue
      
      if is_ornament(sl[0]):
        sl = [0] + sl

      if not is_position(sl[-1]):
        sl = sl + [0]
      
      ls.append(sl)  
      
      st = ed
    
    return ls
  
  groups = []
  split_st = 0
  
  for si in split_idx_list + [None]:
    g = enc_lb[split_st:si]  
    g = clean_findings(g)
    
    for subg in g:
      note = subg[0]
      position = subg[-1]
      ornament = subg[1:-1]
    
      groups.append((note, position, ornament))
    
    if si:
      split_st = si + 1
  
  return groups

for test_case in test_ls:
  label, pred = test_case[2:4]
  
  label_dec = tokenizer.decode( label )
  pred_dec = tokenizer.decode( pred )
  
  label_split = split_enc_seq(tokenizer, label)
  print(label_dec)
  print([ tokenizer.vocab[g[0]] for g in label_split ])
  print([ tokenizer.vocab[g[1]] for g in label_split ])
  print([ tokenizer.vocab[orn] for g in label_split for orn in g[2] ])
  
  pred_split = split_enc_seq(tokenizer, pred)
  print(pred_dec)
  print([ tokenizer.vocab[g[0]] for g in pred_split ])
  print([ tokenizer.vocab[g[1]] for g in pred_split ])
  print([ tokenizer.vocab[orn] for g in pred_split for orn in g[2] ])
  
  print()

청남_노네:2 -:4 니나*:6 청임:8
['청남', '-', '니나*', '청임']
[':2', ':4', ':6', ':8']
['_노네']
청남:2 -:4 니나*:6 청임:8
['청남', '-', '니나*', '청임']
[':2', ':4', ':6', ':8']
[]

고_덧길이표:10 태_반길이표:11
['고', '태']
[':10', ':11']
['_덧길이표', '_반길이표']
고:10 태_반길이표:11
['고', '태']
[':10', ':11']
['_반길이표']

황:2 -:5 태_니레:8
['황', '-', '태']
[':2', ':5', ':8']
['_니레']
황:10 태_니레:11
['황', '태']
[':10', ':11']
['_니레']

황_루러표:5
['황']
[':5']
['_루러표']
황:5
['황']
[':5']
[]

노:2 -:4 니_미는표:6 느나:8
['노', '-', '니', '느나']
[':2', ':4', ':6', ':8']
['_미는표']
노_미는표:2 -:4 니:6 느나:8
['노', '-', '니', '느나']
[':2', ':4', ':6', ':8']
['_미는표']

황:2 -_미는표:5 배중:8
['황', '-', '배중']
[':2', ':5', ':8']
['_미는표']
황_미는표:2 -:5 배중:8
['황', '-', '배중']
[':2', ':5', ':8']
['_미는표']

황_자출:5
['황']
[':5']
['_자출']
황_전성:5
['황']
[':5']
['_전성']

태:2 -:5 -:7 흘림표_니:9
['태', '-', '-', '흘림표']
[':2', ':5', ':7', ':9']
['_니']
태:2 -:5 -:7 니:9
['태', '-', '-', '니']
[':2', ':5', ':7', ':9']
[]

-:2 -_미는표:5 태:8
['-', '-', '태']
[':2', ':5', ':8']
['_미는표']
-:2 -_미는표:5 태_시루표:8
['-', '-', '태

In [15]:
test_lbs = [
  '임_노니로:1 중:3 -:5 무_느니-르_노니로:8_느니-르_노니로:8',
  '황:1 태:3 -:5 -:7 배임_노니로:9_노니로:9',
  '황:1 태:3 -_흘림표:5 -:8 니:9니:9',
  '태:1 황:3 -:4 니:6 배남_흘림표:8_흘림표:8',
  '태:1 황:3 -:4 니:6 배남_흘림표:8_흘림표',
]

for lb in test_lbs:
  print(lb)
  
  lb = tokenizer(lb)[1:-1]  
  lb_split = split_enc_seq(tokenizer, lb)
  print([ tokenizer.vocab[g[0]] for g in lb_split ])
  print([ tokenizer.vocab[g[1]] for g in lb_split ])
  print([ tokenizer.vocab[orn] for g in lb_split for orn in g[2] ])
  
  print()

# print(test_lbs[2])
# get_notes_and_positions(test_lbs[2])

임_노니로:1 중:3 -:5 무_느니-르_노니로:8_느니-르_노니로:8
['임', '중', '-', '무', '<pad>']
[':1', ':3', ':5', ':8', ':8']
['_노니로', '_느니-르', '_노니로', '_느니-르', '_노니로']

황:1 태:3 -:5 -:7 배임_노니로:9_노니로:9
['황', '태', '-', '-', '배임', '<pad>']
[':1', ':3', ':5', ':7', ':9', ':9']
['_노니로', '_노니로']

황:1 태:3 -_흘림표:5 -:8 니:9니:9
['황', '태', '-', '-', '니', '니']
[':1', ':3', ':5', ':8', ':9', ':9']
['_흘림표']

태:1 황:3 -:4 니:6 배남_흘림표:8_흘림표:8
['태', '황', '-', '니', '배남', '<pad>']
[':1', ':3', ':4', ':6', ':8', ':8']
['_흘림표', '_흘림표']

태:1 황:3 -:4 니:6 배남_흘림표:8_흘림표
['태', '황', '-', '니', '배남', '<pad>']
[':1', ':3', ':4', ':6', ':8', '<pad>']
['_흘림표', '_흘림표']



In [16]:
tok_list_decode = lambda tknz, tl: [ tknz.vocab[t] for t in tl ]

def make_category_lists(gl, num_ctg=6): # gl: [(note, pos, [orn])]
  n_l, p_l, np_l, o_l, op_l, onp_l = [ [] for _ in range(num_ctg) ]
  
  for g in gl:
    n, p, oo = g
    
    n_l.append(n)
    p_l.append(p)
    np_l.append((n, p))
    
    o_l += oo
    
    for o in oo:
      op_l.append((o, p))
      onp_l.append((o, n, p))
  
  n_l = filter_specials(n_l)
  p_l = filter_specials(p_l)
  
  return n_l, p_l, np_l, o_l, op_l, onp_l

def prepare_pairs(gt_gl, prd_gl, num_ctg=6): # x_gl: list of tuples
  
  gt_ctg_lists = make_category_lists(gt_gl, num_ctg=num_ctg)
  prd_ctg_lists = make_category_lists(prd_gl, num_ctg=num_ctg)
  
  return list(zip( gt_ctg_lists, prd_ctg_lists ))


for test_case in test_ls[0:1]:
  label, pred = test_case[2:4]
  
  label_dec = tokenizer.decode( label )
  pred_dec = tokenizer.decode( pred )
  
  print(label_dec)
  print(pred_dec)
  
  label_split = split_enc_seq(tokenizer, label)
  pred_split = split_enc_seq(tokenizer, pred)
  
  prepare_pairs(label_split, pred_split)

청남_노네:2 -:4 니나*:6 청임:8
청남:2 -:4 니나*:6 청임:8


In [17]:
def filter_pos_list(pl):
  pl_f = []
  
  for p in pl:
    try:
      int(p) # if str p cannot be cast into integer goto except block
      pl_f.append(p)
    except:
      pass
  
  return pl_f

def calc_metric(gt, prd): # +) [꾸밈](GT 꾸밈 0개 이상일 때), [꾸밈, 위치], [본음, 꾸밈, 위치]
  gt_tok_dict = defaultdict(int)
  
  for tok in gt:
    gt_tok_dict[tok] += 1
  
  num_match = 0
  
  for tok in prd:
    if gt_tok_dict[tok] > 0:
      num_match += 1
      gt_tok_dict[tok] -= 1
  
  eps = 1e-10
  
  precision = num_match / len(prd) if len(prd) > 0 else None
  recall = num_match / len(gt) if len(gt) > 0 else None
  
  f1 = 2 * (precision * recall) / (precision + recall + eps) if precision != None and recall != None else None
  
  return precision, recall, f1

def calc_all_metrics(_tokenizer, gt, prd):
  # gt_dec = _tokenizer.decode( gt )
  # prd_dec = _tokenizer.decode( prd )
  
  gt_split = split_enc_seq(_tokenizer, gt)
  prd_split = split_enc_seq(_tokenizer, prd)
  
  # (note_p, pos_p, note_pos_p, orn_p, orn_pos_p, orn_note_pos_p)
  # ((gt_note, prd_note), (gt_pos, prd_pos), (gt_note_pos, prd_note_pos), (gt_orn, prd_orn), (gt_orn_pos, prd_orn_pos), (gt_orn_note_pos, prd_orn_note_pos))
  pairs = prepare_pairs(gt_split, prd_split)
  
  # (ner, per, nper, oer, oper, onper)
  metric_values = [ calc_metric(*p) for p in pairs]
  
  return pairs, metric_values

test_index = 1
label, pred = test_ls[test_index][2:4]

((label_note, pred_note), (label_pos, pred_pos), (label_note_pos, pred_note_pos), (label_orn, pred_orn), (label_orn_pos, pred_orn_pos), (label_orn_note_pos, pred_orn_note_pos)), (ner, per, nper, oer, oper, onper) = calc_all_metrics(tokenizer, label, pred)

print(ner, calc_metric(label_note, pred_note), label_note, pred_note)
print(per, calc_metric(label_pos, pred_pos), label_pos, pred_pos)
print(nper, calc_metric(label_note_pos, pred_note_pos), label_note_pos, pred_note_pos)
print(oer, calc_metric(label_orn, pred_orn), label_orn, pred_orn)
print(oper, calc_metric(label_orn_pos, pred_orn_pos), label_orn_pos, pred_orn_pos)
print(onper, calc_metric(label_orn_note_pos, pred_orn_note_pos), label_orn_note_pos, pred_orn_note_pos)

(1.0, 1.0, 0.99999999995) (1.0, 1.0, 0.99999999995) [76, 127] [76, 127]
(1.0, 1.0, 0.99999999995) (1.0, 1.0, 0.99999999995) [6, 7] [6, 7]
(1.0, 1.0, 0.99999999995) (1.0, 1.0, 0.99999999995) [(76, 6), (127, 7)] [(76, 6), (127, 7)]
(1.0, 0.5, 0.6666666666222222) (1.0, 0.5, 0.6666666666222222) [52, 58] [58]
(1.0, 0.5, 0.6666666666222222) (1.0, 0.5, 0.6666666666222222) [(52, 6), (58, 7)] [(58, 7)]
(1.0, 0.5, 0.6666666666222222) (1.0, 0.5, 0.6666666666222222) [(52, 76, 6), (58, 127, 7)] [(58, 127, 7)]


## Test new metrics

In [16]:
for idx, test_case in list(enumerate(test_ls))[5:8]:
  label, pred = test_case[2:4]

  label_dec = tokenizer.decode( label )
  pred_dec = tokenizer.decode( pred )

  ( (label_note, pred_note), (label_pos, pred_pos), (label_note_pos, pred_note_pos), (label_orn, pred_orn), (label_orn_pos, pred_orn_pos), (label_orn_note_pos, pred_orn_note_pos) ), (ner, per, nper, oer, oper, onper) = calc_all_metrics(tokenizer, label, pred)
  
  round_tuple_el = lambda t: tuple([ round(el, 4) if el != None else None for el in t ])
  
  ner, per, nper, oer, oper, onper = ( round_tuple_el(mv) for mv in (ner, per, nper, oer, oper, onper) )

  print(f'#{idx} {round(test_case[4], 4)}')
  print(label_dec, len(label_dec))
  print(pred_dec, len(pred_dec))
  print(f'NER: {ner}')
  print(label_note, len(label_note))
  print(pred_note, len(pred_note))
  print(f'PER: {per}')
  print(label_pos, len(label_pos))
  print(pred_pos, len(pred_pos))
  print(f'NPER: {nper}')
  print(label_note_pos, len(label_note_pos))
  print(pred_note_pos, len(pred_note_pos))
  print(f'OER: {oer}')
  print(label_orn, len(label_orn))
  print(pred_orn, len(pred_orn))
  print(f'OPER: {oper}')
  print(label_orn_pos, len(label_orn_pos))
  print(pred_orn_pos, len(pred_orn_pos))
  print(f'ONPER: {onper}')
  print(label_orn_note_pos, len(label_orn_note_pos))
  print(pred_orn_note_pos, len(pred_orn_note_pos))
  print()

#5 0.5556
황:2 -_미는표:5 배중:8 16
황_미는표:2 -:5 배중:8 16
NER: (1.0, 1.0, 1.0)
[140, 4, 106] 3
[140, 4, 106] 3
PER: (1.0, 1.0, 1.0)
[12, 15, 18] 3
[12, 15, 18] 3
NPER: (1.0, 1.0, 1.0)
[(140, 12), (4, 15), (106, 18)] 3
[(140, 12), (4, 15), (106, 18)] 3
OER: (1.0, 1.0, 1.0)
[57] 1
[57] 1
OPER: (0.0, 0.0, 0.0)
[(57, 15)] 1
[(57, 12)] 1
ONPER: (0.0, 0.0, 0.0)
[(57, 4, 15)] 1
[(57, 140, 12)] 1

#6 0.6667
황_자출:5 6
황_전성:5 6
NER: (1.0, 1.0, 1.0)
[140] 1
[140] 1
PER: (1.0, 1.0, 1.0)
[15] 1
[15] 1
NPER: (1.0, 1.0, 1.0)
[(140, 15)] 1
[(140, 15)] 1
OER: (0.0, 0.0, 0.0)
[67] 1
[68] 1
OPER: (0.0, 0.0, 0.0)
[(67, 15)] 1
[(68, 15)] 1
ONPER: (0.0, 0.0, 0.0)
[(67, 140, 15)] 1
[(68, 140, 15)] 1

#7 0.75
태:2 -:5 -:7 흘림표_니:9 19
태:2 -:5 -:7 니:9 15
NER: (0.75, 0.75, 0.75)
[127, 4, 4, 141] 4
[127, 4, 4, 87] 4
PER: (1.0, 1.0, 1.0)
[12, 15, 17, 19] 4
[12, 15, 17, 19] 4
NPER: (0.75, 0.75, 0.75)
[(127, 12), (4, 15), (4, 17), (141, 19)] 4
[(127, 12), (4, 15), (4, 17), (87, 19)] 4
OER: (None, 0.0, None)
[47] 1
[] 0
OPER: (

In [18]:
num_metrics = 6
metric_ls = [ [] for _ in range(num_metrics) ] # ner, per, nper, oer, oper, onper

for case in pred_list:
  label, pred = case[2:4]
  
  _, metric_values = calc_all_metrics(tokenizer, label, pred)
  
  for m_idx, mv in enumerate(metric_values):
    metric_ls[m_idx].append(mv)

print(len(pred_list))
print(len(metric_ls)) # should be 6
print(len(metric_ls[1])) # should be the same as len(pred_list)
print(len(metric_ls[0][0])) # 3
print()

avg_metric_ls = []

for mv_ls in metric_ls:
  avg_mv = []
  
  for subm_i in range(3):
    subm_ls = [ subm_t[subm_i] for subm_t in mv_ls if subm_t[subm_i] != None ]
    avg_subm = sum(subm_ls) / len(subm_ls)
    avg_mv.append( avg_subm )
  
  avg_metric_ls.append(tuple(avg_mv))

print(len(pred_list))
print(len(avg_metric_ls)) # should be 6
print(len(avg_metric_ls[1])) # 3
print()

metric_names = ('ner', 'per', 'nper', 'oer', 'oper', 'opner')
for m_name, avg_mv in zip(metric_names, avg_metric_ls):
  print(m_name, avg_mv)

1532
6
1532
3

1532
6
3

ner (0.982909051348999, 0.9915578764142733, 0.9860182390363174)
per (0.9830178416013925, 0.9896866840731069, 0.9854825294601532)
nper (0.9736401218450828, 0.9817558746736292, 0.9769360896802187)
oer (0.9779827798277979, 0.9449799196787148, 0.9752067824153218)
oper (0.9402624026240257, 0.9088353413654616, 0.9376964432936863)
opner (0.9365723657236568, 0.9040160642570279, 0.9335607939969812)


## Test Pipeline

In [20]:
def test(project_root_dir, exp_dir, csv_path):
  conf = OmegaConf.load(exp_dir / '.hydra' / 'config.yaml')
  device = torch.device(conf.general.device)
  
  model_dir = exp_dir / 'model'
  
  print('\n...tokenizer loading...')
  
  tokenizer_vocab_fn = model_dir / f'{conf.general.model_name}_tokenizer.txt'
  tokenizer = Tokenizer(vocab_txt_fn=tokenizer_vocab_fn)
  
  print('COMPLETE: load tokenizer')
  
  
  print('\n...data_set loading...')
  
  test_set = LabelStudioDataset(project_root_dir / conf.data_path.test, project_root_dir / 'jeongganbo-png/splited-pngs', remove_borders=conf.test_setting.remove_borders, is_valid=True)
  
  test_set.tokenizer = tokenizer
  
  test_loader = DataLoader(test_set, batch_size=1000, shuffle=False, collate_fn=pad_collate, num_workers=conf.dataloader.num_workers_load)
  
  print('COMPLETE: data_set loading')
  
  
  print('\n...model initializing...')
  model = TransformerOMR(conf.model.dim, len(tokenizer.vocab), enc_depth=conf.model.enc_depth, dec_depth=conf.model.dec_depth, num_heads=conf.model.num_heads, dropout=conf.model.dropout)
  model.load_state_dict(torch.load(model_dir / f'{conf.general.model_name}_HL_{conf.test_setting.target_metric}_best.pt', map_location='cpu')['model'])

  tester = Trainer(model, 
                  None, #optimizer
                  None, #loss_fn
                  None, #train_loader
                  test_loader, 
                  tokenizer,
                  device=device, 
                  scheduler=None,
                  aux_loader=None,
                  aux_freq=None,
                  mix_aux=None,
                  aux_valid_loader=None,
                  wandb=None, 
                  model_name=conf.general.model_name,
                  model_save_path=model_dir,
                  checkpoint_logger=None)

  print('COMPLETE: model initializing')
  

  print('\n...testing...')
  
  _, metric_dict, pred_tensor_list, _ = tester.validate(with_confidence=True)
  
  print('COMPLETE: Testing')
  
  
  print('\n...processing test result...')
  
  pred_list = []

  for b_idx in range(len(pred_tensor_list)):
    pred_list += pred_tensor_list[b_idx].tolist()

  pred_list = list( enumerate(pred_list) )
  pred_list = make_gt_pred_tuple_list(tokenizer, pred_list)
  
  print('COMPLETE: processing test result')
  
  print('\n...calculating metrics...')
  
  num_metrics = 6
  metric_ls = [ [] for _ in range(num_metrics) ] # ner, per, nper, oer, oper, onper
  
  for case in pred_list:
    label, pred = case[2:4]
    
    _, metric_values = calc_all_metrics(tokenizer, label, pred)
    
    for m_idx, mv in enumerate(metric_values):
      metric_ls[m_idx].append(mv)
  
  print(len(pred_list))
  print(len(metric_ls)) # should be 6
  print(len(metric_ls[1])) # should be the same as num_total_case
  print(len(metric_ls[0][0])) # 3
  print()
  
  avg_metric_ls = []
  
  for mv_ls in metric_ls:
    avg_mv = []
    
    for subm_i in range(3):
      subm_ls = [ subm_t[subm_i] for subm_t in mv_ls if subm_t[subm_i] != None ]
      avg_subm = sum(subm_ls) / len(subm_ls)
      avg_mv.append( avg_subm )
    
    avg_metric_ls.append(tuple(avg_mv))

  print(len(pred_list))
  print(len(avg_metric_ls)) # should be 6
  print(len(avg_metric_ls[1])) # 3
  print()
  
  exact_all = metric_dict[conf.test_setting.target_metric]
  
  metric_names = ('ner', 'per', 'nper', 'oer', 'oper', 'opner')
  for m_name, avg_mv in zip(metric_names, avg_metric_ls):
    print(m_name, avg_mv)
  
  with open(csv_path, 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # writer.writerow([conf.general.model_name, metric_dict[conf.test_setting.target_metric]] + metric_values)
    writer.writerow( [conf.general.model_name, exact_all] + [mv_t[2] for mv_t in avg_metric_ls] )
  
  print('COMPLETE: calculating metrics')

project_root_dir = Path('.')
output_dir = project_root_dir / 'outputs'
dir_date = '2024-05-04'
dir_time = '20-31-38'

exp_dir = output_dir / dir_date / dir_time  

csv_path = project_root_dir / 'test' / 'new_metric_test.csv'

pred_list = test(project_root_dir, exp_dir, csv_path)


...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing...
COMPLETE: model initializing

...testing...


COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
1532
6
1532
3

1532
6
3

ner (0.982909051348999, 0.9915578764142733, 0.9860182390363174)
per (0.9830178416013925, 0.9896866840731069, 0.9854825294601532)
nper (0.9736401218450828, 0.9817558746736292, 0.9769360896802187)
oer (0.9779827798277979, 0.9449799196787148, 0.9752067824153218)
oper (0.9402624026240257, 0.9088353413654616, 0.9376964432936863)
opner (0.9365723657236568, 0.9040160642570279, 0.9335607939969812)
COMPLETE: calculating metrics


## Test

In [3]:
from datetime import datetime
from test_er import test as test_func

get_ts = lambda: datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

project_root_dir = Path('.')
output_dir = project_root_dir / 'outputs'
save_path = project_root_dir / 'test'

In [4]:
# model paths
dir_dates = ['2024-05-03', '2024-05-04']
dir_times = [
  [
    '21-55-11', 
    '22-41-31', 
    '23-27-48', 
  ],
  [
    '00-14-10', 
    '01-01-26', 
    '01-48-46', 
    '02-36-04', 
    '03-22-26', 
    '04-08-48', 
    '04-55-03', 
    '05-42-17', 
    '06-29-30', 
    '07-16-44', 
    '08-02-58', 
    '08-49-17', 
    '09-35-37', 
    '10-22-54', 
    '11-10-10', 
    '11-57-33', 
    '12-43-58', 
    '13-30-12', 
    '14-16-32', 
    '15-03-40', 
    '15-50-51', 
    '16-38-09', 
    '17-24-27', 
    '18-10-43', 
    '18-57-02', 
    '19-44-22',     
    '20-31-38', 
  ]
]

In [5]:
csv_path = save_path / 'new_metric_test.csv'

dir_date, dir_time = (dir_dates[1], dir_times[1][-1])

exp_dir = output_dir / dir_date / dir_time  
test_func(project_root_dir, exp_dir, csv_path)


...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing...
COMPLETE: model initializing

...testing...


COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.982909051348999, 0.9915578764142733, 0.9860182390363174)
per (0.9830178416013925, 0.9896866840731069, 0.9854825294601532)
nper (0.9736401218450828, 0.9817558746736292, 0.9769360896802187)
oer (0.9792127921279209, 0.9461847389558232, 0.9764474772043417)
oper (0.9414924149241487, 0.9100401606425701, 0.9389371380827061)
opner (0.9378023780237797, 0.9052208835341363, 0.934801488786001)
EMR (None, None, 0.8674934725848564)
NMR (None, None, 0.9321148825065274)
PMR (None, None, 0.9497389033942559)
OrMR (None, None, 0.9425587467362925)
dist (None, None, 0.3133159268929504)
dist/len (None, None, 0.03863266798347821)
insert (None, None, 0.07963446475195822)
del (None, None, 0.17558746736292427)
sub (None, None, 0.058093994778067884)
COMPLETE: calculating metrics


In [6]:
csv_path = save_path / f'metric_test_{get_ts()}.csv'

with open(csv_path, 'a', newline='', encoding='utf-8') as f:
  writer = csv.writer(f)
  writer.writerow( ('name', 'EM', 'ner', 'per', 'nper', 'oer', 'oper', 'opner', 'EMR', 'NMR', 'PMR', 'OrMR', 'dist', 'dist/len', 'ins', 'del', 'sub') )

for date_idx, dir_date in enumerate(dir_dates):
  for dir_time in dir_times[date_idx]:
    
    exp_dir = output_dir / dir_date / dir_time  
    test_func(project_root_dir, exp_dir, csv_path)


...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing...
COMPLETE: model initializing

...testing...


COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.984508268059182, 0.9798629242819845, 0.9810364460977339)
per (0.9848999129677987, 0.9814186248912096, 0.9823008148880727)
nper (0.9711270670147963, 0.9679503916449087, 0.9689254065717541)
oer (0.9163274536408861, 0.7853413654618474, 0.9350902183750829)
oper (0.9020805065581181, 0.7732931726907631, 0.9206077872266898)
opner (0.8932609678878332, 0.7660642570281125, 0.9118233617760173)
EMR (None, None, 0.7721932114882507)
NMR (None, None, 0.9379895561357703)
PMR (None, None, 0.95822454308094)
OrMR (None, None, 0.8198433420365535)
dist (None, None, 0.40404699738903394)
dist/len (None, None, 0.05560255612618915)
insert (None, None, 0.23172323759791122)
del (None, None, 0.07180156657963446)
sub (None, None, 0.10052219321148825)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.9770778938207137, 0.9777088772845952, 0.9763346820946595)
per (0.9805700609225414, 0.9819516971279372, 0.980677391169587)
nper (0.9619669277632725, 0.9630113141862489, 0.9620322489610553)
oer (0.8199191473722893, 0.8926706827309239, 0.9388676031871191)
oper (0.8019110621095182, 0.875401606425703, 0.9200510855209957)
opner (0.7949283351708927, 0.866967871485944, 0.9119625372029795)
EMR (None, None, 0.7826370757180157)
NMR (None, None, 0.935378590078329)
PMR (None, None, 0.95822454308094)
OrMR (None, None, 0.8178851174934726)
dist (None, None, 0.4112271540469974)
dist/len (None, None, 0.07605899237369049)
insert (None, None, 0.10966057441253264)
del (None, None, 0.1768929503916449)
sub (None, None, 0.12467362924281984)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing...
C

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.98371409921671, 0.978959965187119, 0.978993402834116)
per (0.9722584856396871, 0.9730744125326368, 0.9716795895807607)
nper (0.9589969538729333, 0.9611074847693646, 0.9595767917407502)
oer (0.9535960378983633, 0.8779116465863458, 0.95009648298875)
oper (0.931847545219638, 0.8596385542168677, 0.9297478991116338)
opner (0.9182816537467697, 0.8475903614457834, 0.9166760036874294)
EMR (None, None, 0.8322454308093995)
NMR (None, None, 0.9268929503916449)
PMR (None, None, 0.9412532637075718)
OrMR (None, None, 0.8890339425587467)
dist (None, None, 0.38642297650130547)
dist/len (None, None, 0.05130862850779144)
insert (None, None, 0.17558746736292427)
del (None, None, 0.10966057441253264)
sub (None, None, 0.10117493472584857)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing...


COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.9856940818102699, 0.9912206266318536, 0.9872427180675923)
per (0.9849543080939946, 0.9909268929503915, 0.9868014817581774)
nper (0.9769038294168841, 0.9816253263707572, 0.9784386667375722)
oer (0.8457255747126438, 0.9366465863453817, 0.9601485148023513)
oper (0.8279454022988506, 0.9157630522088355, 0.9394389438462326)
opner (0.8230962643678161, 0.9097389558232933, 0.9338696369158282)
EMR (None, None, 0.8257180156657964)
NMR (None, None, 0.9595300261096605)
PMR (None, None, 0.9634464751958225)
OrMR (None, None, 0.8550913838120104)
dist (None, None, 0.3198433420365535)
dist/len (None, None, 0.06521365517151496)
insert (None, None, 0.060704960835509136)
del (None, None, 0.18407310704960836)
sub (None, None, 0.07506527415143603)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializ

...

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.9765665796344647, 0.988239773716275, 0.980313344134432)
per (0.9749129677980855, 0.9822998259355965, 0.9773214238123022)
nper (0.9646214099216713, 0.9733681462140997, 0.9677457623113432)
oer (0.8384393063583816, 0.8661646586345381, 0.9459417011701062)
oper (0.8218689788053951, 0.8503012048192773, 0.9280876994119318)
opner (0.8168593448940269, 0.8458835341365463, 0.9229425806786135)
EMR (None, None, 0.7669712793733682)
NMR (None, None, 0.9210182767624021)
PMR (None, None, 0.9412532637075718)
OrMR (None, None, 0.8204960835509139)
dist (None, None, 0.5176240208877284)
dist/len (None, None, 0.0932039481470253)
insert (None, None, 0.11553524804177545)
del (None, None, 0.29569190600522194)
sub (None, None, 0.10639686684073107)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing.

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.9869560487380331, 0.9819081810269805, 0.983084057610044)
per (0.9716818973020018, 0.9718885987815489, 0.971130975879605)
nper (0.961303307223673, 0.9619451697127939, 0.9612375149747022)
oer (0.9451476793248945, 0.8822289156626508, 0.9372498935233594)
oper (0.910337552742616, 0.850100401606426, 0.9030225627458802)
opner (0.9052742616033754, 0.8452811244979923, 0.8981268624481225)
EMR (None, None, 0.8466057441253264)
NMR (None, None, 0.9392950391644909)
PMR (None, None, 0.9497389033942559)
OrMR (None, None, 0.8981723237597912)
dist (None, None, 0.3681462140992167)
dist/len (None, None, 0.045947927234884)
insert (None, None, 0.16057441253263707)
del (None, None, 0.0783289817232376)
sub (None, None, 0.12924281984334204)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing...
CO

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.9856505657093124, 0.9878698868581374, 0.9856257464114787)
per (0.9797867711053091, 0.9828764142732812, 0.9807088508616234)
nper (0.970659268929504, 0.9734660574412537, 0.9716069214596501)
oer (0.8683765501691093, 0.8979919678714858, 0.9456963162139117)
oper (0.8396279594137542, 0.8664658634538154, 0.9136208445173482)
opner (0.8351183765501691, 0.8604417670682731, 0.9083797543708843)
EMR (None, None, 0.8133159268929504)
NMR (None, None, 0.9484334203655352)
PMR (None, None, 0.9608355091383812)
OrMR (None, None, 0.8537859007832899)
dist (None, None, 0.37467362924281983)
dist/len (None, None, 0.06254854901664142)
insert (None, None, 0.11945169712793734)
del (None, None, 0.15274151436031333)
sub (None, None, 0.10248041775456919)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializi

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.9882801815243067, 0.9911771105308964, 0.988917102574846)
per (0.9883703220191471, 0.9911227154046995, 0.9891306763523444)
nper (0.9805343155538977, 0.9825935596170583, 0.9811865896878095)
oer (0.8900038226299695, 0.9162650602409639, 0.961652555046031)
oper (0.8745221712538227, 0.8993975903614457, 0.9444184230586767)
opner (0.8706995412844036, 0.8945783132530121, 0.9402149762325017)
EMR (None, None, 0.8433420365535248)
NMR (None, None, 0.9614882506527415)
PMR (None, None, 0.9686684073107049)
OrMR (None, None, 0.8727154046997389)
dist (None, None, 0.2800261096605744)
dist/len (None, None, 0.05122246783617924)
insert (None, None, 0.08942558746736293)
del (None, None, 0.1325065274151436)
sub (None, None, 0.058093994778067884)
COMPLETE: calculating metrics

...tokenizer loading...
COMPLETE: load tokenizer

...data_set loading...
COMPLETE: data_set loading

...model initializing

COMPLETE: Testing

...processing test result...
COMPLETE: processing test result

...calculating metrics...
ner (0.982909051348999, 0.9915578764142733, 0.9860182390363174)
per (0.9830178416013925, 0.9896866840731069, 0.9854825294601532)
nper (0.9736401218450828, 0.9817558746736292, 0.9769360896802187)
oer (0.9792127921279209, 0.9461847389558232, 0.9764474772043417)
oper (0.9414924149241487, 0.9100401606425701, 0.9389371380827061)
opner (0.9378023780237797, 0.9052208835341363, 0.934801488786001)
EMR (None, None, 0.8674934725848564)
NMR (None, None, 0.9321148825065274)
PMR (None, None, 0.9497389033942559)
OrMR (None, None, 0.9425587467362925)
dist (None, None, 0.3133159268929504)
dist/len (None, None, 0.03863266798347821)
insert (None, None, 0.07963446475195822)
del (None, None, 0.17558746736292427)
sub (None, None, 0.058093994778067884)
COMPLETE: calculating metrics
